# Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# [Load] 230206_RH_4등급DB세분류_인증번호변경 건_v1.2

In [2]:
fold = 'D:/data/big2/4등급_배인번호_재확인'

In [17]:
file_name = '4등급DB세분류_인증번호변경 건_2302091522.csv'
df = pd.read_csv(os.path.join(fold, file_name), encoding='cp949', low_memory=False)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61359 entries, 0 to 61358
Data columns (total 32 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   등급(현재)     61359 non-null  object 
 1   말소(현재)     61359 non-null  object 
 2   인증번호(현재)   61359 non-null  object 
 3   인증(검증)     0 non-null      float64
 4   인증번호(변경전)  0 non-null      float64
 5   인증번호(변경후)  58995 non-null  object 
 6   차대번호       61359 non-null  object 
 7   차량번호       61359 non-null  object 
 8   제원관리번호     61359 non-null  object 
 9   차종         61359 non-null  object 
 10  용도         61359 non-null  object 
 11  최초등록일      61359 non-null  int64  
 12  연식         61359 non-null  int64  
 13  제작일자       25275 non-null  float64
 14  검사유효일      61358 non-null  float64
 15  등급         61359 non-null  object 
 16  차명         61359 non-null  object 
 17  차종분류       61359 non-null  object 
 18  차종유형       61359 non-null  object 
 19  자동차형식      61352 non-null  object 
 20  제작사명  

In [19]:
df.columns

Index(['등급(현재)', '말소(현재)', '인증번호(현재)', '인증(검증)', '인증번호(변경전)', '인증번호(변경후)',
       '차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량',
       '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부'],
      dtype='object')

# Check data

## 기초

In [20]:
df.isnull().sum()

등급(현재)           0
말소(현재)           0
인증번호(현재)         0
인증(검증)       61359
인증번호(변경전)    61359
인증번호(변경후)     2364
차대번호             0
차량번호             0
제원관리번호           0
차종               0
용도               0
최초등록일            0
연식               0
제작일자         36084
검사유효일            1
등급               0
차명               0
차종분류             0
차종유형             0
자동차형식            7
제작사명          1034
연료               0
엔진형식           266
총중량              0
적재중량            22
엔진출력            88
배기량              0
법정동코드            0
시도명              0
시군구명             0
소유자구분            0
말소여부             0
dtype: int64

In [21]:
df['등급(현재)'].value_counts(dropna=False)

4     60658
5       677
T4       12
3        12
Name: 등급(현재), dtype: int64

In [22]:
df['등급'].value_counts(dropna=False)

4     60658
5       676
T4       13
3        12
Name: 등급, dtype: int64

In [23]:
df['말소(현재)'].value_counts(dropna=False)

N    59295
Y     2064
Name: 말소(현재), dtype: int64

In [24]:
df['말소여부'].value_counts(dropna=False)

N    61359
Name: 말소여부, dtype: int64

## 5등급 배인번호 데이터

In [25]:
g5 = df[df['등급(현재)'] == '5']
g5.shape

(677, 32)

In [26]:
g5['인증번호(현재)'].unique()

array(['2MY-DW-04', '6MY-SS-0M-24', '5MY-HD-24-39', '5MY-KM-24-32',
       '9MY-KM-14-41', '5MY-SY-24-48', '7MY-DB-24-44'], dtype=object)

### [로드] 5등급 등록&제원정보(말소제거)

In [15]:
ods_fold = 'D:/data/big2/ODS'

In [16]:
# about 
file_name = '[ODS][G5]등록정보&제원정보(말소제거)(2022.12.29).csv'
info = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
info.shape

(1154762, 49)

In [33]:
info.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [28]:
# about 42s
for one in tqdm(g5['인증번호(현재)'].unique()):
    with pd.ExcelWriter(f'C:/Users/kbjung/Documents/GitHub/wabotech/car_big_data_2/analysis2/4등급_데이터_점검/recheck/{one}.xlsx') as writer:
        temp = info[info['배출가스인증번호'] == one].reset_index(drop=True)
        temp.to_excel(writer, index=False)

100%|██████████| 7/7 [00:41<00:00,  5.95s/it]


In [29]:
info['제작사명'].unique()

array(['현대자동차(주)', '거화', nan, '지엠코리아(주)', '기아자동차(주)', '지엠아시아퍼시픽지역본부 주식회사',
       '새한자동차', '대호무역', '**기타**', '쌍용자동차(주)', '자기인증면제차량', '한국토요타자동차(주)',
       '비엠더블유코리아(주)', 'GM', '포드세일즈서비스코리아(유)', '대우자동차(주)', '한국지엠주식회사',
       '(주)미래하이텍', '신진자동차', '아시아자동차공업(주)', 'TOYOTA', '-', 'BENZ',
       '한성자동차(주)', '동아자동차', '후소특장(주)', '남도수입', '메르세데스벤츠코리아(주)', '(주)기아정기',
       '혼다코리아(주)', '에프씨에이코리아(주)', '볼보', 'MITSUBISHI', 'BMW AG', 'FORD',
       '한국상용차(주)', '광림특장차(주)', '스카니아', '다임러크라이슬러(주)', '대우중공업(주)',
       '세화통운(주)', '타타대우상용차(주)', '자동차수입판매업', '포르쉐', '서울차량공업(주)', '현대정공(주)',
       '남영자동차공업(주)', '폭스바겐', 'SOFA차량', '로젠바워', '금양기전', '극동유화(주) 고진모터 임포츠',
       '(주)수산특장', '삼성중공업', '현대', 'General Trailers France S.A.',
       '지피모터스(주)', '삼성상용차(주)', '신정개발', '한국특장(주)', '경기특장개발(주)',
       '두성기계공업(주)', '(주)한빛특장', '설봉강업(주)', '삼일자동차(주)', '(주)우리특장',
       '다임폴라특장(주)', '덕산특장기공사', '신흥공업(주)', '(주)한성에코넷', '서울차체공업(주)', '볼보트럭',
       '(주)한원모터스', '대흥중공업(주)', '도요다', '랜드로버', '해글런드비히클', '스폐셜리티비히클',
       '부커구이어', 

In [36]:
info.loc[(info['제작사명'] == '자일대우상용차 주식회사') & (info['차명'] == 'BH120'), ['배출가스인증번호', '차량연식','차명', '자동차형식', '엔진형식']]

,배출가스인증번호,차량연식,차명,자동차형식,엔진형식
542345,3MY-DB-06,2003,BH120,BH120-E4,DV15TIS
545739,3MY-DB-06,2003,BH120,BH120-E2,DV15TIS
547469,3MY-DB-06,2003,BH120,BH120-E2,DV15TIS
548849,3MY-DB-06,2003,BH120,BH120-E2,DV15TIS
549414,3MY-DB-06,2003,BH120,BH120-E27,DV15TIS
...,...,...,...,...,...
1121159,4MY-DB-03,2007,BH120,BH120-GP7R4S332S,DV11
1123524,4MY-DB-03,2007,BH120,BH120-GP7R4S412S,DV11
1125590,4MY-DB-03,2007,BH120,BH120-GP7R4S452S,DV11
1152674,4MY-DB-03,2007,BH120,BH120-GP7R4S452S,DV11


In [37]:
info.loc[(info['제작사명'] == '자일대우상용차 주식회사') & (info['차명'] == 'BH120'), '차량연식'].unique()

array([2003, 2004, 2005, 2006, 2007], dtype=int64)

#### [샘플] 7MY-DB-24-44과 미매칭, 동일 제작사, 동일 차명

In [38]:
info.loc[(info['제작사명'] == '자일대우상용차 주식회사') & (info['차명'] == 'BH120')].to_excel('analysis2/4등급_데이터_점검/recheck/7MY-DB-24-44_미매칭_동일제작사_동일차명_샘플_말소제거.xlsx', index=False)

### [로드] 5등급 등록&제원정보

In [39]:
# about 
file_name = '[ODS][G5]등록정보&제원정보(2022.12.29).csv'
info = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
info.shape

(2378648, 49)

In [40]:
info.loc[(info['제작사명'] == '자일대우상용차 주식회사') & (info['차명'] == 'BH120'), '차량연식'].unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009], dtype=int64)

In [41]:
info.loc[(info['제작사명'] == '자일대우상용차 주식회사') & (info['차명'] == 'BH120')].to_excel('analysis2/4등급_데이터_점검/recheck/7MY-DB-24-44_미매칭_동일제작사_동일차명_샘플.xlsx', index=False)

# Code End